In [1]:
import pandas as pd 
import numpy as np 

from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_excel("preparedData.xlsx")

In [3]:
data.head()

,date,new_cases,new_cases_smoothed,reproduction_rate,new_tests,positive_rate,tests_per_case,total_vaccinations,people_vaccinated,people_fully_vaccinated,...,book covid test: (Ireland)_y,covid test dublin: (Ireland),covid test centre: (Ireland),hse covid vaccine: (Ireland),hse vaccine portal: (Ireland),hse portal vaccine: (Ireland),pcr test hse: (Ireland),hse covid test: (Ireland),hse vaccine registration: (Ireland),how long will it take to vaccinate ireland: (Ireland)
0,2020-03-15,0,15.714,2.51,0,0.0,0.0,0,0,0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,2020-03-16,40,21.143,2.59,0,0.0,0.0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2020-03-17,54,27.000,2.70,0,0.0,0.0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2020-03-18,69,35.571,2.78,0,0.0,0.0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2020-03-19,265,73.429,2.84,946,0.0,0.0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [4]:
data['date'] = pd.to_datetime(data['date'])
data.sort_values(by='date', ascending=True, inplace=True)
data = data.set_index('date')

In [5]:
data_new_cases = data[['new_cases_smoothed']]

In [8]:
period = int(input("Enter the days in advance to predict: "))

Enter the days in advance to predict: 5


In [9]:
data_new_cases.loc[:,'minus'] = data_new_cases.loc[:,'new_cases_smoothed'].shift(periods=period)

In [11]:
data_new_cases.loc[:,'minus_diff'] = data_new_cases.loc[:,'minus'].diff()

In [12]:
data_new_cases = data_new_cases.dropna()

In [13]:
data_new_cases.head()

,new_cases_smoothed,minus,minus_diff
date,,,
2020-03-21,93.714,21.143,5.429
2020-03-22,111.000,27.000,5.857
2020-03-23,136.571,35.571,8.571
2020-03-24,158.000,73.429,37.858
2020-03-25,181.714,84.714,11.285


In [14]:
data_new_cases.shape

(696, 3)

In [15]:
n = len(data_new_cases)

In [16]:
variables = data_new_cases.columns.drop('new_cases_smoothed')

In [17]:
X_train = data_new_cases.iloc[0:int(n*0.7)].drop(['new_cases_smoothed'], axis=1)
y_train = data_new_cases.iloc[0:int(n*0.7)].drop(columns=variables, axis=1)

In [18]:
X_test = data_new_cases.iloc[int(n*0.9):].drop(['new_cases_smoothed'], axis=1)
y_test = data_new_cases.iloc[int(n*0.9):].drop(columns=variables, axis=1)

In [19]:
X_val = data_new_cases.iloc[int(n*0.7):int(n*0.9)].drop(['new_cases_smoothed'], axis=1)
y_val = data_new_cases.iloc[int(n*0.7):int(n*0.9)].drop(columns=variables, axis=1)

In [20]:
model = MLPRegressor()

In [21]:
param_search = {
    "hidden_layer_sizes": [(1,),(50,),(100,),(150,),(200,)],
    "activation": ["identity", "logistic", "tanh", "relu"],
    "solver": ["lbfgs", "sgd", "adam"],
    "alpha": [0.00005,0.0005, 0.005],
    "learning_rate": ['constant', 'invscaling', 'adaptive']
}

In [22]:
tsvc = TimeSeriesSplit(n_splits=2)
gsearch = GridSearchCV(estimator=model, cv=tsvc, param_grid=param_search, scoring='r2')
gsearch.fit(X_train, y_train)
best_score = gsearch.best_score_
best_model = gsearch.best_estimator_

In [23]:
print(best_model)

MLPRegressor(activation='identity', alpha=5e-05, hidden_layer_sizes=(1,),
             solver='lbfgs')


In [24]:
print(best_score)

0.7670612387172857


In [25]:
from sklearn.metrics import r2_score

y_pred = best_model.predict(X_test)

print(r2_score(y_test, y_pred))

0.5164691326706712


In [26]:
y_val_pred = best_model.predict(X_val)

In [27]:
print(r2_score(y_val, y_val_pred))

0.7987093448150469
